In [24]:
# 문제 정의 : 의료서비스 밀집 지역과 아닌 지역을 도출해 고령층이 많이 거주하는 지역에 의료 시설이 충분히 제공되고 있는지 확인하고자 합니다.

# 예상 결과 : 전라도와 강원도 지역이 인구 밀도와 인프라 밀집도가 낮아 의료 수요가 부족할 것으로 예상됩니다.

In [ ]:
# 데이터 선택 : 전국 병원정보 데이터, 전국 약국 정보 데이터, 전국 지역별 연령별 총 인구수 데이터를 통해
# ???선택한 데이터셋의 구조와 해당 데이터가 어떻게 문제 해결에 도움을 줄 수 있는지 작성하기???

In [1]:
# 필요한 라이브러리 불러오기

import pandas as pd #데이터 프레임을 다루는 라이브러리
import numpy as np #수치계산을 하기 위한 라이브러리
import matplotlib.pyplot as plt #데이터 시각화를 위한 라이브러리
from sklearn.cluster import KMeans #kmeans 클러스터링 알고리즘을 위한 라이브러리
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [31]:
# 데이터 로드
df1 = pd.read_csv("1.병원정보서비스 2024.3.csv")
df2 = pd.read_csv("2.약국정보서비스 2024.3.csv")
df3 = pd.read_csv("3.지역별 연령별 총 인구수.csv", encoding='CP949')

In [37]:
# 1번 전국 병원 정보 데이터가 로드 됐는지 확인
df1.head()

,암호화요양기호,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,우편번호,...,치과인턴 인원수,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,조산사 인원수,좌표(X),좌표(Y)
0,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교 성빈센트병원,1,상급종합,310000,경기,310603,수원팔달구,지동,16247,...,0,0,7,0,0,0,0,0,127.027427,37.277985
1,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교인천성모병원,1,상급종합,220000,인천,220003,인천부평구,부평동,21431,...,0,0,2,0,0,0,0,0,126.724899,37.484831
2,JDQ4MTYyMiM1MSMkMSMkNCMkODkkMzgxMzUxIzExIyQxIy...,강릉아산병원,1,상급종합,320000,강원,320100,강릉시,사천면,25440,...,0,0,2,0,0,0,0,0,128.857841,37.818433
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,강북삼성병원,1,상급종합,110000,서울,110016,종로구,평동,03181,...,0,0,4,0,0,0,0,0,126.967750,37.568408
4,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,건국대학교병원,1,상급종합,110000,서울,110023,광진구,화양동,05030,...,0,0,2,0,0,0,0,0,127.071828,37.540376


In [35]:
# 2번 전국 약국 정보 데이터가 로드 됐는지 확인
df2.head()

,암호화요양기호,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,우편번호,주소,전화번호,개설일자,좌표(X),좌표(Y)
0,JDQ4MTg4MSM1MSMkMSMkMCMkMDMkNTgxOTYxIzQxIyQxIy...,이층약국,81,약국,110000,서울,110015,은평구,역촌동,03421,"서울특별시 은평구 연서로 9, 2층 205(상가)호 (역촌동, 센타폴리스)",02-6085-0466,2018.2.7,126.915668,37.600118
1,JDQ4MTg4MSM1MSMkMSMkNCMkMDMkNDgxNzAyIzUxIyQyIy...,한림약국,81,약국,110000,서울,110002,강동구,천호동,05339,"서울특별시 강동구 성안로 149, 1층 (천호동)",02-486-9743,2023.3.27,127.134721,37.536590
2,JDQ4MTg4MSM1MSMkMSMkNCMkMDMkNDgxNzAyIzUxIyQyIy...,21세기상계약국,81,약국,110000,서울,110022,노원구,상계동,01674,"서울특별시 노원구 동일로 1548, (상계동)",937-2288,2005.4.1,127.057642,37.666680
3,JDQ4MTYyMiM3MSMkMSMkMCMkMDMkMzgxNzAyIzMxIyQxIy...,정문약국,81,약국,360000,전남,360022,화순군,화순읍,58127,"전라남도 화순군 화순읍 서양로 313, (화순읍)",061-372-7077,2024.1.12,127.000395,35.059973
4,JDQ4MTYyMiM1MSMkMSMkMCMkMDMkNDgxMTkxIzExIyQxIy...,알찬약국,81,약국,310000,경기,310402,성남중원구,금광동,13179,"경기도 성남시 중원구 산성대로 382, 1층 (금광동)",NaN,2024.1.15,127.157902,37.445920


In [36]:
# 3번 전국 지역별 연령별 정보 데이터가 로드 됐는지 확인
df3.head()

,행정구역,2024년03월_계_총인구수,2024년03월_계_연령구간인구수,2024년03월_계_0~9세,2024년03월_계_10~19세,2024년03월_계_20~29세,2024년03월_계_30~39세,2024년03월_계_40~49세,2024년03월_계_50~59세,2024년03월_계_60~69세,...,2024년03월_여_10~19세,2024년03월_여_20~29세,2024년03월_여_30~39세,2024년03월_여_40~49세,2024년03월_여_50~59세,2024년03월_여_60~69세,2024년03월_여_70~79세,2024년03월_여_80~89세,2024년03월_여_90~99세,2024년03월_여_100세 이상
0,전국 (0000000000),"51,293,934","51,293,934","3,276,777","4,637,563","6,130,341","6,562,325","7,877,433","8,717,794","7,692,991",...,"2,252,146","2,930,936","3,144,800","3,871,483","4,320,987","3,905,731","2,171,765","1,305,284","238,736","7,214"
1,서울특별시 (1100000000),"9,384,739","9,384,739","504,805","733,051","1,348,801","1,426,416","1,412,882","1,496,980","1,321,940",...,"358,539","710,454","712,927","713,718","761,150","696,278","409,170","205,078","34,186","1,154"
2,부산광역시 (2600000000),"3,287,292","3,287,292","191,016","263,231","372,557","389,014","481,477","541,515","561,841",...,"128,351","181,334","188,509","238,220","276,894","297,290","178,344","89,372","13,767",375
3,대구광역시 (2700000000),"2,369,962","2,369,962","148,196","212,813","277,660","279,784","353,187","420,314","368,979",...,"103,185","131,268","130,632","178,123","215,188","192,426","110,285","62,621","9,943",232
4,인천광역시 (2800000000),"3,006,045","3,006,045","200,674","271,951","360,948","415,021","480,309","517,390","447,137",...,"132,415","174,058","198,600","233,230","259,644","226,283","107,770","60,900","12,028",387


In [44]:
# 결측치처리, 데이터정규화, 변수 선택 등의 전처리 작업하기

#결측치처리 : 결측치가 있는 열을 제거
df1.dropna(axis=1, inplace=True)
df2.dropna(axis=1, inplace=True)
df3.dropna(axis=1, inplace=True)
df1.head()

,암호화요양기호,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,주소,개설일자,...,의과전문의 인원수,치과일반의 인원수,치과인턴 인원수,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,조산사 인원수
0,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교 성빈센트병원,1,상급종합,310000,경기,310603,수원팔달구,"경기도 수원시 팔달구 중부대로 93, (지동)",1967.5.23,...,238,0,0,0,7,0,0,0,0,0
1,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교인천성모병원,1,상급종합,220000,인천,220003,인천부평구,"인천광역시 부평구 동수로 56, (부평동)",1981.8.6,...,235,1,0,0,2,0,0,0,0,0
2,JDQ4MTYyMiM1MSMkMSMkNCMkODkkMzgxMzUxIzExIyQxIy...,강릉아산병원,1,상급종합,320000,강원,320100,강릉시,"강원특별자치도 강릉시 사천면 방동길 38, (사천면)",1996.7.30,...,159,0,0,0,2,0,0,0,0,0
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,강북삼성병원,1,상급종합,110000,서울,110016,종로구,"서울특별시 종로구 새문안로 29, (평동)",1979.3.24,...,251,4,0,0,4,0,0,0,0,0
4,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,건국대학교병원,1,상급종합,110000,서울,110023,광진구,"서울특별시 광진구 능동로 120-1, (화양동)",1982.11.16,...,233,0,0,0,2,0,0,0,0,0


In [26]:
#k-means 클러스터링 알고리즘을 사용해 데이터를 클러스터링 하기

In [27]:
# 엘보우 방법을 사용해 최적의 클러스터 수 k를 결정하고 클러 수에 따른 wcss를 그래프로 나타내기

In [28]:
# 결정된 k값으로 k-means 알고리즘을 초기화 하고 클러스터링을 수행하기

In [29]:
# 결과 분석 및 평가
#클러스터 분석 : 
#각 클러스터의 중심점을 분석하고 중심점을 바탕으로 각 클러스터의 특성 설명하기
#클러스터를 시각화해 각 클러스터가 어떻게 구성되어있는지 보여주는 산점도 작성

In [30]:
#평가 : 실루엣 점수를 사용해 클러스터링 효과를 평가 / 실루엣 점수가 클러스터링 결과에 대해 어떤 정보를 제공하는지 설명